In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3" ,
    task="text-generation",
    max_new_tokens=100,
    do_sample=False,
)
llm.invoke("Hugging Face is")

In [ ]:
print(llm.invoke("Please provide 5 multiple choice questions related to the subject of Indian economy"))

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader(r"C:\Users\imvbh\Desktop\LLMs\hface\langchain\data\Langchain\Components\pdf_files")

In [ ]:
documents = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# split the extracted data into text chunks using the text_splitter, which splits the text based on the specified number of characters and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)
# print the number of chunks obtained
len(text_chunks)

In [ ]:
text_chunks

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

In [ ]:
from langchain_chroma import Chroma
cdb = Chroma.from_documents(documents, embedding_model)


In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                 retriever=cdb.as_retriever(search_kwargs={"k": 1}))

# define a query to ask the system
query = "How do we use text splitter in Langchain ?"
# run the system and get a response
qa.invoke(query)


In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFDirectoryLoader(r"C:\Users\imvbh\Desktop\LLMs\hface\langchain\data\Langchain\LangChain Expression Language\pdf_files")

documents = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# split the extracted data into text chunks using the text_splitter, which splits the text based on the specified number of characters and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)
# print the number of chunks obtained
len(text_chunks)

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
repo_id ="mistralai/Mistral-7B-Instruct-v0.2"
#repo_id = "imvbhuvan/aspireai-7b-V0.6-4bit"  
gen_llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature= 0.75, model_kwargs= {'max_length': 8192}, max_new_tokens=4096)

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
repo_id ="mistralai/Mistral-7B-Instruct-v0.3"
#repo_id = "imvbhuvan/aspireai-7b-V0.6-4bit"  
cric_llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature= 0.75, model_kwargs= {'max_length': 8192}, max_new_tokens=4096)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

generator_llm = Ollama(model="phi3")
critic_llm = Ollama(model="llama3")

ollama_emb = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.from_langchain(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=ollama_emb
)

# generate testset
testset = generator.generate_with_langchain_docs(text_chunks, test_size=50, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25}, raise_exceptions=False)

# Try using this methond to load llms

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

# Getting responses from our RAG pipeline

In [ ]:
from datasets import Dataset
from tqdm.auto import tqdm
import pandas as pd


def generate_response(query_engine, question):
    response = query_engine.query(question)
    return {
        "answer": response.response,
        "contexts": [c.node.get_content() for c in response.source_nodes],
    }


def generate_ragas_dataset(query_engine, test_df):
    test_questions = test_df["question"].values
    responses = [generate_response(query_engine, q) for q in tqdm(test_questions)]

    dataset_dict = {
        "question": test_questions,
        "answer": [response["answer"] for response in responses],
        "contexts": [response["contexts"] for response in responses],
        "ground_truth": test_df["ground_truth"].values.tolist(),
    }
    ds = Dataset.from_dict(dataset_dict)
    return ds

ragas_eval_dataset = generate_ragas_dataset(query_engine, test_df)
ragas_evals_df = pd.DataFrame(ragas_eval_dataset)
ragas_evals_df.head(1)

# Getting scores 

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [answer_correctness]

critic_lm = Critic()

ollama_emb = OllamaEmbeddings(
    model="nomic-embed-text",
)

evaluation_result = evaluate(
    llm= critic_lm,
    embeddings=ollama_emb,
    dataset=ragas_eval_dataset,
    metrics=metrics
)
eval_scores_df = pd.DataFrame(evaluation_result.scores)
eval_scores_df.head(5)